In [207]:
import pandas as pd
import numpy as np
import utm
import matplotlib.pyplot as plt
from utils.preprocessing import preprocess
from utils.from_latlon import from_latlon
import math
from sklearn.linear_model import LinearRegression  # Import LinearRegression class



In [208]:
data_type = ['Simulated BTFS', 'BTFS']
freq = '3min'
routine = 'training'
dimensions = ['xOffset', 'yOffset']
sm = 150 # maximum signal difference between min and max, used for calibration

In [209]:
def preprocess_sim_data(sim_data, data_type, freq, tower_locs, routine):
    # Get data
    sim_dat_filt = sim_data[sim_data['Data_type'].isin(data_type)]
    
    sim_dat_filt, predictors = preprocess(sim_dat_filt, freq, routine)
     
    # Calculate easting and northing from lat long
    sim_dat_filt['easting'], sim_dat_filt['northing'], sim_dat_filt['zone_num'], sim_dat_filt['zone_letter'] = from_latlon(sim_dat_filt['POINT_Y'].values, sim_dat_filt['POINT_X'].values)

    # Create a dictionary of the coordinates of the towers
    offset_dict = tower_locs.set_index('TowerID').to_dict()
    point_x = offset_dict['POINT_X']
    point_y = offset_dict['POINT_Y']

    # Standardise the coordinates so that the tower location == 0 on both the x and y axes.
    sim_dat_filt['xOffset'] = sim_dat_filt['easting'] - sim_dat_filt['TowerID'].map(point_x).fillna(0)
    sim_dat_filt['yOffset'] = sim_dat_filt['northing'] - sim_dat_filt['TowerID'].map(point_y).fillna(0)
    
    sim_dat_filt['easting_of_tower'] = sim_dat_filt['TowerID'].map(point_x).fillna(0)
    sim_dat_filt['northing_of_tower'] = sim_dat_filt['TowerID'].map(point_y).fillna(0)

    return sim_dat_filt, predictors

In [210]:
test_data = 'Example_data\Output\Train_test_data\Testing_Tag_GPS_locations.xlsx'

# Get testing data
test_data = pd.read_excel(test_data)
test_data['DateAndTime'] = pd.to_datetime(test_data['DateAndTime'])

# Get training data
train_data = 'Example_data\Output\Train_test_data\Training_Tag_GPS_locations.xlsx'
train_data = pd.read_excel(train_data)
train_data['DateAndTime'] = pd.to_datetime(test_data['DateAndTime'])

# Get tower locations
radio_tower_xy_path = 'Example_data\Input\Radio_tower_locations\RTEastNorth.xlsx'
tower_locs = pd.read_excel(radio_tower_xy_path)
train_data['DateAndTime'] = pd.to_datetime(train_data['DateAndTime'])

# Get ML model location estimates (for comparison)
ml_estimates_path = r'Example_data\Output\Predictions\UTM_predictions_combined_dtypes_input_20230701.xlsx'
ml_estimates = pd.read_excel(ml_estimates_path)

In [211]:
# Create test dataset as per train_model_h2o method
test_data_preproc, predictors_test = preprocess_sim_data(test_data, data_type, freq, tower_locs, routine)
train_data_preproc, predictors_train = preprocess_sim_data(train_data, data_type, freq, tower_locs, routine)

In [212]:
def calculate_delta_g(s1, s2, sm):
    delta_g = (s1-s2)/sm
    return delta_g

def calculate_offset_angle(delta_g):
    bearing = math.acos(delta_g) * 90/math.pi
    return bearing

In [213]:
# Make distance values absolute
train_data_preproc['xOffset_abs'] = train_data_preproc['xOffset'].abs()
train_data_preproc['yOffset_abs'] = train_data_preproc['yOffset'].abs()

# Perform linear regression for each antenna
train_data_filtered = train_data_preproc[train_data_preproc['ant1_mean'] != 0]
x = train_data_filtered[['ant1_mean']]
y = train_data_filtered[['yOffset_abs']]
model_ant1 = LinearRegression().fit(x, y)

train_data_filtered = train_data_preproc[train_data_preproc['ant2_mean'] != 0]
x = train_data_filtered[['ant2_mean']]
y = train_data_filtered[['xOffset_abs']]
model_ant2 = LinearRegression().fit(x, y)

train_data_filtered = train_data_preproc[train_data_preproc['ant3_mean'] != 0]
x = train_data_filtered[['ant3_mean']]
y = train_data_filtered[['yOffset_abs']]
model_ant3 = LinearRegression().fit(x, y)

train_data_filtered = train_data_preproc[train_data_preproc['ant4_mean'] != 0]
x = train_data_filtered[['ant4_mean']]
y = train_data_filtered[['xOffset_abs']]
model_ant4 = LinearRegression().fit(x, y)

# Also create an average model
# Extract coefficients from individual models
coefficients_ant1 = np.array([model_ant1.coef_[0][0], model_ant1.intercept_[0]])
coefficients_ant2 = np.array([model_ant2.coef_[0][0], model_ant2.intercept_[0]])
coefficients_ant3 = np.array([model_ant3.coef_[0][0], model_ant3.intercept_[0]])
coefficients_ant4 = np.array([model_ant4.coef_[0][0], model_ant4.intercept_[0]])

# Calculate average coefficients
average_coefficients = np.mean([coefficients_ant1, coefficients_ant2, coefficients_ant3, coefficients_ant4], axis=0)

# Create a new Linear Regression model with average coefficients
model_ant_average = LinearRegression()
model_ant_average.coef_ = np.array([average_coefficients[0]])
model_ant_average.intercept_ = np.array([average_coefficients[1]])

In [214]:
## Calculate total paired antenna values to find maximum sum, only if both values do not equal zero.

# Add 'ant_1_2_sum' column
test_data_preproc['ant_1_2_sum'] = test_data_preproc.apply(lambda row: row['ant1_mean'] + row['ant2_mean'] if row['ant1_mean'] != 0 and row['ant2_mean'] != 0 else 'na', axis=1)

# Add 'ant_2_3_sum' column
test_data_preproc['ant_2_3_sum'] = test_data_preproc.apply(lambda row: row['ant2_mean'] + row['ant3_mean'] if row['ant2_mean'] != 0 and row['ant3_mean'] != 0 else 'na', axis=1)

# Add 'ant_3_4_sum' column
test_data_preproc['ant_3_4_sum'] = test_data_preproc.apply(lambda row: row['ant3_mean'] + row['ant4_mean'] if row['ant3_mean'] != 0 and row['ant4_mean'] != 0 else 'na', axis=1)

# Add 'ant_4_1_sum' column
test_data_preproc['ant_4_1_sum'] = test_data_preproc.apply(lambda row: row['ant4_mean'] + row['ant1_mean'] if row['ant4_mean'] != 0 and row['ant1_mean'] != 0 else 'na', axis=1)



In [215]:
#### Calculate the bearings from the tower

# List of antenna names
antenna_names = ['ant_1_2_sum', 'ant_2_3_sum', 'ant_3_4_sum', 'ant_4_1_sum']

# Create a new column 'bearing' in the DataFrame
test_data_preproc['bearing'] = None

# Iterate through the DataFrame
for index, row in test_data_preproc.iterrows():
    # Extract values from the new columns and handle 'na' values
    values = [float(row['ant_1_2_sum']) if row['ant_1_2_sum'] != 'na' else float('-inf'),
              float(row['ant_2_3_sum']) if row['ant_2_3_sum'] != 'na' else float('-inf'),
              float(row['ant_3_4_sum']) if row['ant_3_4_sum'] != 'na' else float('-inf'),
              float(row['ant_4_1_sum']) if row['ant_4_1_sum'] != 'na' else float('-inf')]
    
    # Check if all values are 'na'; if yes, skip the row
    if all(val == float('-inf') for val in values):
        continue
    
    # Find the column name with the highest value
    max_column = max(zip(values, antenna_names))[1]
        
    if max_column == "ant_1_2_sum":
        if row.ant1_mean >= row.ant2_mean:
            delta_g = calculate_delta_g(row.ant1_mean, row.ant2_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)
            bearing = offset_angle
            
            # print(f'delta_g = {delta_g}')
            # print(f'offset_angle from ant 1 = {offset_angle}')
            # print(f'bearing ant 1 = {bearing}')

        else:
            delta_g = calculate_delta_g(row.ant2_mean, row.ant1_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)          
            bearing = 90 - offset_angle

    elif max_column == "ant_2_3_sum":
        if row.ant2_mean >= row.ant3_mean:
            delta_g = calculate_delta_g(row.ant2_mean, row.ant3_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)
            bearing = 90 + offset_angle

        else:
            delta_g = calculate_delta_g(row.ant3_mean, row.ant2_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)          
            bearing = 180 - offset_angle

    elif max_column == "ant_3_4_sum":
        if row.ant3_mean >= row.ant4_mean:
            delta_g = calculate_delta_g(row.ant3_mean, row.ant4_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)
            bearing = 180 + offset_angle

        else:
            delta_g = calculate_delta_g(row.ant4_mean, row.ant3_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)          
            bearing = 270 - offset_angle

    elif max_column == "ant_4_1_sum":
        if row.ant4_mean >= row.ant1_mean:
            delta_g = calculate_delta_g(row.ant4_mean, row.ant1_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)
            bearing = 270 + offset_angle

        else:
            delta_g = calculate_delta_g(row.ant1_mean, row.ant4_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)          
            bearing = 360 - offset_angle
    
    
    # Assign the calculated bearing to the 'bearing' column
    test_data_preproc.at[index, 'bearing'] = bearing

In [216]:
# Convert non-numeric values to NaN
numeric_columns = ['ant_1_2_sum', 'ant_2_3_sum', 'ant_3_4_sum', 'ant_4_1_sum']
test_data_preproc[numeric_columns] = test_data_preproc[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Calculate the maximum value across columns while handling NaN values
test_data_preproc['max_value'] = np.nanmax(test_data_preproc[numeric_columns].values, axis=1) / 2


C:\Users\s5236256\AppData\Local\Temp\ipykernel_6752\4233910366.py:6: RuntimeWarning: All-NaN slice encountered
  test_data_preproc['max_value'] = np.nanmax(test_data_preproc[numeric_columns].values, axis=1) / 2


In [217]:
test_data_preproc

,DateTime,TowerID,TagID,Data_type,POINT_X,POINT_Y,Point_ID,Tag_type,Interval_seconds,ant1_count,...,xOffset,yOffset,easting_of_tower,northing_of_tower,ant_1_2_sum,ant_2_3_sum,ant_3_4_sum,ant_4_1_sum,bearing,max_value
0,2021-02-02 07:51:00,RT01,60,BTFS,146.256427,-21.919968,299,Nanotag,13,0.0,...,-135.520989,-73.105345,423338.731184,7.575918e+06,NaN,NaN,NaN,NaN,None,NaN
1,2021-02-02 07:51:00,RT04,60,BTFS,146.256427,-21.919968,299,Nanotag,13,1.0,...,257.092334,171.458690,422946.117861,7.575674e+06,NaN,NaN,NaN,NaN,None,NaN
2,2021-02-02 07:54:00,RT01,60,BTFS,146.256427,-21.919968,299,Nanotag,13,0.0,...,-135.520989,-73.105345,423338.731184,7.575918e+06,NaN,NaN,NaN,NaN,None,NaN
3,2021-02-02 07:54:00,RT04,60,BTFS,146.256427,-21.919968,299,Nanotag,13,3.0,...,257.092334,171.458690,422946.117861,7.575674e+06,NaN,NaN,255.916667,265.666667,317.741648,132.833333
4,2021-02-04 18:36:00,RT12,52,BTFS,146.295603,-22.016401,303,Nanotag,13,2.0,...,53.909953,-458.008627,427244.674540,7.565648e+06,NaN,NaN,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,2023-05-23 10:42:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,15.0,...,103.510404,-165.520759,421129.737817,7.573987e+06,211.200000,225.133333,216.990476,203.057143,136.36288,112.566667
612,2023-05-23 10:45:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,15.0,...,103.510404,-165.520759,421129.737817,7.573987e+06,207.747619,224.521978,222.450549,205.676190,137.31218,112.260989
613,2023-05-23 10:48:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,14.0,...,103.510404,-165.520759,421129.737817,7.573987e+06,210.285714,226.928571,210.595238,193.952381,135.559351,113.464286
614,2023-05-23 10:51:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,11.0,...,103.510404,-165.520759,421129.737817,7.573987e+06,190.204545,200.750000,193.800000,183.254545,136.002881,100.375000


In [218]:
# Calculate locations without using biangulation

# List of antenna names
antennas = ['ant1_mean', 'ant2_mean', 'ant3_mean', 'ant4_mean']

for index, row in test_data_preproc.iterrows():
    # First calculate appropriate bearing and infer distance using linear model
    if row['bearing'] != None:
        distance = model_ant_average.predict(np.array([[row['max_value']]]))[0]
        bearing = row['bearing']
        test_data_preproc.at[index, 'Mech_method'] = "Multiple antenna, single tower"

    elif row['bearing'] == None:
        # Extract values from the new columns and handle 'na' values
        values = [float(row['ant1_mean']) if row['ant1_mean'] != 'na' else float('-inf'),
                float(row['ant2_mean']) if row['ant2_mean'] != 'na' else float('-inf'),
                float(row['ant3_mean']) if row['ant3_mean'] != 'na' else float('-inf'),
                float(row['ant4_mean']) if row['ant4_mean'] != 'na' else float('-inf')]

        test_data_preproc.at[index, 'Mech_method'] = "Single antenna, single tower"

        # Find the column name with the highest value
        max_single_ant = max(zip(values, antennas))[1]

        if max_single_ant == "ant1_mean":
            bearing = 0
            distance = model_ant1.predict(np.array([[row['ant1_mean']]]))[0]

        elif max_single_ant == "ant2_mean":
            bearing = 90
            distance = model_ant2.predict(np.array([[row['ant2_mean']]]))[0]

        elif max_single_ant == "ant3_mean":
            bearing = 180
            distance = model_ant3.predict(np.array([[row['ant3_mean']]]))[0]

        elif max_single_ant == "ant4_mean":
            bearing = 270
            distance = model_ant4.predict(np.array([[row['ant4_mean']]]))[0]

    else:
        pass

    # Estimating new location using trigonometry
    bearing_rad = math.radians(bearing)  # Convert bearing to radians

    # Calculate new coordinates
    new_easting = row['easting_of_tower'] + distance * math.sin(bearing_rad)
    new_northing = row['northing_of_tower'] + distance * math.cos(bearing_rad)

    # Assigning the new coordinates to the 'new_easting' and 'new_northing' columns in test_data_preproc
    test_data_preproc.at[index, 'mech_bearing_easting'] = new_easting
    test_data_preproc.at[index, 'mech_bearing_northing'] = new_northing



c:\Users\s5236256\Documents\GitHub\ml4rt\.venv\Lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\s5236256\Documents\GitHub\ml4rt\.venv\Lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\s5236256\Documents\GitHub\ml4rt\.venv\Lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\s5236256\Documents\GitHub\ml4rt\.venv\Lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\s5236256\Documents\GitHub\ml4rt\.venv\Lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature nam

In [219]:
test_data_preproc

,DateTime,TowerID,TagID,Data_type,POINT_X,POINT_Y,Point_ID,Tag_type,Interval_seconds,ant1_count,...,northing_of_tower,ant_1_2_sum,ant_2_3_sum,ant_3_4_sum,ant_4_1_sum,bearing,max_value,Mech_method,mech_bearing_easting,mech_bearing_northing
0,2021-02-02 07:51:00,RT01,60,BTFS,146.256427,-21.919968,299,Nanotag,13,0.0,...,7.575918e+06,NaN,NaN,NaN,NaN,None,NaN,"Single antenna, single tower",423440.771919,7.575918e+06
1,2021-02-02 07:51:00,RT04,60,BTFS,146.256427,-21.919968,299,Nanotag,13,1.0,...,7.575674e+06,NaN,NaN,NaN,NaN,None,NaN,"Single antenna, single tower",422946.117861,7.575821e+06
2,2021-02-02 07:54:00,RT01,60,BTFS,146.256427,-21.919968,299,Nanotag,13,0.0,...,7.575918e+06,NaN,NaN,NaN,NaN,None,NaN,"Single antenna, single tower",423186.963655,7.575918e+06
3,2021-02-02 07:54:00,RT04,60,BTFS,146.256427,-21.919968,299,Nanotag,13,3.0,...,7.575674e+06,NaN,NaN,255.916667,265.666667,317.741648,132.833333,"Multiple antenna, single tower",422883.361853,7.575743e+06
4,2021-02-04 18:36:00,RT12,52,BTFS,146.295603,-22.016401,303,Nanotag,13,2.0,...,7.565648e+06,NaN,NaN,NaN,NaN,None,NaN,"Single antenna, single tower",427244.674540,7.565400e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,2023-05-23 10:42:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,15.0,...,7.573987e+06,211.200000,225.133333,216.990476,203.057143,136.36288,112.566667,"Multiple antenna, single tower",421249.358485,7.573862e+06
612,2023-05-23 10:45:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,15.0,...,7.573987e+06,207.747619,224.521978,222.450549,205.676190,137.31218,112.260989,"Multiple antenna, single tower",421248.081934,7.573859e+06
613,2023-05-23 10:48:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,14.0,...,7.573987e+06,210.285714,226.928571,210.595238,193.952381,135.559351,113.464286,"Multiple antenna, single tower",421248.624533,7.573866e+06
614,2023-05-23 10:51:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,11.0,...,7.573987e+06,190.204545,200.750000,193.800000,183.254545,136.002881,100.375000,"Multiple antenna, single tower",421283.581419,7.573828e+06


In [221]:
# Replace 'None' and 'na' with NaN in the 'bearing' column
test_data_preproc['bearing'].replace(['None', 'na'], float('nan'), inplace=True)

# Drop rows with any missing values in 'bearing' column
test_data_preproc_nona = test_data_preproc.dropna(subset=['bearing'], how='any')

In [222]:
def triang(x1, y1, alpha1, x2, y2, alpha2):
    # For Triangulation GK Coordinates are necessary!
    # First calculate tan keeping in mind that 0° in geo-coordinates are 90° in an x-y plane
    ta1 = (alpha1 % 360) / 180 * np.pi
    ta2 = (alpha2 % 360) / 180 * np.pi

    if ((alpha1 - alpha2) % 180 == 0):
        # print("No triangulation possible: all three points are on one line")
        return (np.nan, np.nan)

    # Finding Intersection Using solver
    b = np.array([x2 - x1, y2 - y1])
    a1 = np.array([np.sin(ta1), np.cos(ta1)])
    a2 = np.array([-np.sin(ta2), -np.cos(ta2)])
    a = np.vstack((a1, a2))
    
    try:
        l = np.linalg.solve(a, b)
    except np.linalg.LinAlgError:
        return (np.nan, np.nan)

    px = x1 + l[0] * np.sin(ta1)
    py = y1 + l[0] * np.cos(ta1)

    if l[1] > 0 and l[0] > 0:
        return px, py
    else:
        return np.nan, np.nan



In [223]:
def apply_triangulation(df):
    result_list = []

    # Group by 'TagID' and 'DateTime'
    grouped = df.groupby(['TagID', 'DateTime'])

    for name, group in grouped:
        # For each unique pair of 'TowerID'
        tower_ids = group['TowerID'].unique()

        # Skip if there's only one tower or missing data
        if len(tower_ids) < 2 or group['bearing'].isnull().any():
            continue

        # Generate all unique pairs of TowerID
        tower_combinations = np.array(np.meshgrid(tower_ids, tower_ids)).T.reshape(-1, 2)

        for towers in tower_combinations:
            # Extract data for each TowerID pair
            data1 = group[group['TowerID'] == towers[0]]
            data2 = group[group['TowerID'] == towers[1]]

            # Extract x, y, and alpha values
            x1, y1, alpha1 = data1['easting_of_tower'].values[0], data1['northing_of_tower'].values[0], data1['bearing'].values[0]
            x2, y2, alpha2 = data2['easting_of_tower'].values[0], data2['northing_of_tower'].values[0], data2['bearing'].values[0]

            # Apply triangulation
            easting_estimated, northing_estimated = triang(x1, y1, alpha1, x2, y2, alpha2)
            
            # Append the result along with TagID, DateTime, and TowerID information
            result_list.append({'TagID': name[0], 'DateTime': name[1], 'TowerID1': towers[0], 'TowerID2': towers[1],
                                'mech_tri_easting': easting_estimated, 'mech_tri_northing': northing_estimated})

    # Create a DataFrame from the results
    result_df = pd.DataFrame(result_list)

    return result_df

# Apply triangulation to your data
result_dataframe = apply_triangulation(test_data_preproc_nona)

# Print or use result_dataframe as needed
print(result_dataframe)


     TagID            DateTime TowerID1 TowerID2  mech_tri_easting  \
0        8 2021-04-23 15:12:00     RT01     RT01               NaN   
1        8 2021-04-23 15:12:00     RT01     RT05               NaN   
2        8 2021-04-23 15:12:00     RT05     RT01     423006.319555   
3        8 2021-04-23 15:12:00     RT05     RT05               NaN   
4       50 2021-02-10 16:09:00     RT01     RT01               NaN   
..     ...                 ...      ...      ...               ...   
247    209 2023-05-17 10:33:00     RT15     RT15               NaN   
248    214 2023-05-20 10:09:00     RT01     RT01               NaN   
249    214 2023-05-20 10:09:00     RT01     RT02               NaN   
250    214 2023-05-20 10:09:00     RT02     RT01               NaN   
251    214 2023-05-20 10:09:00     RT02     RT02               NaN   

     mech_tri_northing  
0                  NaN  
1                  NaN  
2         7.576232e+06  
3                  NaN  
4                  NaN  
..       

In [229]:
# Merge the triangulation data back in with the bearing estimates
mech_bearing_and_tri_estimates = pd.merge(test_data_preproc, mech_pos_estimates, on=['TagID', 'DateTime'], how='left')


In [230]:
mech_bearing_and_tri_estimates

,DateTime,TowerID,TagID,Data_type,POINT_X,POINT_Y,Point_ID,Tag_type,Interval_seconds,ant1_count,...,ant_2_3_sum,ant_3_4_sum,ant_4_1_sum,bearing,max_value,Mech_method,mech_bearing_easting,mech_bearing_northing,mech_tri_easting,mech_tri_northing
0,2021-02-02 07:51:00,RT01,60,BTFS,146.256427,-21.919968,299,Nanotag,13,0.0,...,NaN,NaN,NaN,NaN,NaN,"Single antenna, single tower",423440.771919,7.575918e+06,NaN,NaN
1,2021-02-02 07:51:00,RT04,60,BTFS,146.256427,-21.919968,299,Nanotag,13,1.0,...,NaN,NaN,NaN,NaN,NaN,"Single antenna, single tower",422946.117861,7.575821e+06,NaN,NaN
2,2021-02-02 07:54:00,RT01,60,BTFS,146.256427,-21.919968,299,Nanotag,13,0.0,...,NaN,NaN,NaN,NaN,NaN,"Single antenna, single tower",423186.963655,7.575918e+06,NaN,NaN
3,2021-02-02 07:54:00,RT04,60,BTFS,146.256427,-21.919968,299,Nanotag,13,3.0,...,NaN,255.916667,265.666667,317.741648,132.833333,"Multiple antenna, single tower",422883.361853,7.575743e+06,NaN,NaN
4,2021-02-04 18:36:00,RT12,52,BTFS,146.295603,-22.016401,303,Nanotag,13,2.0,...,NaN,NaN,NaN,NaN,NaN,"Single antenna, single tower",427244.674540,7.565400e+06,429709.251945,7.563588e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,2023-05-23 10:42:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,15.0,...,225.133333,216.990476,203.057143,136.362880,112.566667,"Multiple antenna, single tower",421249.358485,7.573862e+06,NaN,NaN
612,2023-05-23 10:45:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,15.0,...,224.521978,222.450549,205.676190,137.312180,112.260989,"Multiple antenna, single tower",421248.081934,7.573859e+06,NaN,NaN
613,2023-05-23 10:48:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,14.0,...,226.928571,210.595238,193.952381,135.559351,113.464286,"Multiple antenna, single tower",421248.624533,7.573866e+06,NaN,NaN
614,2023-05-23 10:51:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,11.0,...,200.750000,193.800000,183.254545,136.002881,100.375000,"Multiple antenna, single tower",421283.581419,7.573828e+06,NaN,NaN


In [231]:
# Excel data export to work out why bearings could not be calculated

mech_bearing_and_tri_estimates.to_excel('Example_data\Output\Predictions\mech_raw_bearings_working_20231111.xlsx', index=False)

In [234]:
# Group by 'TagID' and 'DateTime' and calculate the mean of easting and northing first for the bearing estimates
mech_pos_estimates = mech_bearing_and_tri_estimates.groupby(['TagID', 'DateTime']).agg({'mech_bearing_easting': 'mean', 'mech_bearing_northing': 'mean', 'mech_tri_easting': 'mean', 'mech_tri_northing': 'mean'}).reset_index()

In [236]:
# Merge result_dataframe with ml_estimates
merged_dataframe = pd.merge(ml_estimates, mech_pos_estimates, on=['TagID', 'DateTime'], how='left')

merged_dataframe['mech_tri_easting_error'] = merged_dataframe['mech_tri_easting'] - merged_dataframe['easting']
merged_dataframe['mech_tri_northing_error'] = merged_dataframe['mech_tri_northing'] - merged_dataframe['northing']

merged_dataframe['mech_bearing_easting_error'] = merged_dataframe['mech_bearing_easting'] - merged_dataframe['easting']
merged_dataframe['mech_bearing_northing_error'] = merged_dataframe['mech_bearing_northing'] - merged_dataframe['northing']

# Calculate the Eucledian distance between the predicted and actual locations
merged_dataframe['error_mech_tri_m'] = np.sqrt((merged_dataframe['mech_tri_easting_error']) ** 2
                        + (merged_dataframe['mech_tri_northing_error']) ** 2)

merged_dataframe['error_mech_bearing_m'] = np.sqrt((merged_dataframe['mech_bearing_easting_error']) ** 2
                        + (merged_dataframe['mech_bearing_northing_error']) ** 2)

merged_dataframe

,DateTime,TagID,easting,northing,easting_pred,northing_pred,Tower_count,Data_type,Signal_count,xOffset,...,mech_bearing_easting,mech_bearing_northing,mech_tri_easting,mech_tri_northing,mech_tri_easting_error,mech_tri_northing_error,mech_bearing_easting_error,mech_bearing_northing_error,error_mech_tri_m,error_mech_bearing_m
0,2021-02-02 07:51:00,60,423203.210195,7.575845e+06,423128.694243,7.575921e+06,2,Tracked bird,4,60.785672,...,423193.444890,7.575870e+06,NaN,NaN,NaN,NaN,-9.765305,24.408604,NaN,26.289563
1,2021-02-02 07:54:00,60,423203.210195,7.575845e+06,423065.351077,7.575734e+06,2,Tracked bird,15,60.785672,...,423035.162754,7.575831e+06,NaN,NaN,NaN,NaN,-168.047441,-14.642300,NaN,168.684141
2,2021-02-04 18:36:00,52,427298.584493,7.565190e+06,427436.823929,7.565282e+06,4,Tracked bird,31,-168.960554,...,427491.997383,7.565331e+06,429709.251945,7.563588e+06,2410.667452,-1601.962361,193.412890,140.606772,2894.40857,239.120911
3,2021-02-04 18:39:00,52,427298.584493,7.565190e+06,427376.232446,7.565197e+06,4,Tracked bird,10,-168.960554,...,427423.235942,7.565253e+06,NaN,NaN,NaN,NaN,124.651449,62.931452,NaN,139.636498
4,2021-02-10 16:09:00,50,423203.210195,7.575845e+06,423398.945954,7.575982e+06,4,Simulated bird,20,48.473880,...,423359.252782,7.575957e+06,NaN,NaN,NaN,NaN,156.042587,111.784491,NaN,191.950674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,2023-05-23 10:42:00,192,421233.248221,7.573822e+06,421073.286576,7.574000e+06,1,Tracked bird,58,103.510404,...,421249.358485,7.573862e+06,NaN,NaN,NaN,NaN,16.110263,40.069528,NaN,43.186893
734,2023-05-23 10:45:00,192,421233.248221,7.573822e+06,421073.286576,7.574001e+06,1,Tracked bird,56,103.510404,...,421248.081934,7.573859e+06,NaN,NaN,NaN,NaN,14.833712,37.217753,NaN,40.064950
735,2023-05-23 10:48:00,192,421233.248221,7.573822e+06,421073.286576,7.573993e+06,1,Tracked bird,54,103.510404,...,421248.624533,7.573866e+06,NaN,NaN,NaN,NaN,15.376312,44.289815,NaN,46.883031
736,2023-05-23 10:51:00,192,421233.248221,7.573822e+06,421073.286576,7.573991e+06,1,Tracked bird,44,103.510404,...,421283.581419,7.573828e+06,NaN,NaN,NaN,NaN,50.333197,6.195015,NaN,50.713006


In [237]:
# Excel data export
merged_dataframe.to_excel('Example_data\Output\Predictions\mech_ml_estimates_20231111.xlsx', index=False)